## Imports

In [2]:
import numpy as np
import pandas as pd
import jsonlines
import gc
import torch
from torch import nn

from IPython.display import display, clear_output

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers.activations import GELUActivation
from transformers.modeling_outputs import MaskedLMOutput
from transformers import DataCollatorForWholeWordMask
from datasets import load_from_disk, load_dataset
from transformers import BertTokenizer, DistilBertTokenizer
from transformers.data.data_collator import _torch_collate_batch
import evaluate

import wandb
# wandb.init(project="kg-lm-integration", entity="tanny411")

from huggingface_hub import notebook_login
# notebook_login()

emb_tsv_file = "wikidata_translation_v1.tsv"

In [3]:
import sys
print(sys.getrecursionlimit())

sys.setrecursionlimit(100000)
# but doing so is dangerous -- the standard limit is a little conservative, but Python stackframes can be quite big.

3000


In [4]:
# torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.get_device_name(0)

In [5]:
bert_model_name = "distilbert-base-uncased" ##"bert-base-cased"

# Initialize Project

Download:
- embeds_wktxt.csv
- [linked-wikitext-2 dataset](https://rloganiv.github.io/linked-wikitext-2/#/) and unzip

# Tokenization

- `tokens` are the given list of tokens from wikitext2
- `input_ids` are what come from tokenization, they divide certain words into multiple pieces, and each sentence has a CLS and a SEP
- `word_tokens` is the length of `tokens`. For each token in `token`, it mentions how many sub-words it was divided into due to word piece tokenization
- `cummulative_word_tokens` is a cummulative sum of `word_tokens`, with an extra 0 in the beginning

##### Process
index of a token in `token` can be found in `input_ids` by `cummulative_word_tokens`. if `ix` is the index of a word in `token`, its beginning index in `input_ids` is `cummulative_word_tokens[ix] + 1`, the +1 is because `input_ids` has a CLS in the beginning. `token[ix]` spans from `input_ids[cummulative_word_tokens[ix] + 1]` to `input_ids[cummulative_word_tokens[ix+1] + 1]`

In [6]:
embeds_wktxt = pd.read_csv("embeds_wktxt.csv")

linked_wikitext_2 = "linked-wikitext-2/"
train = linked_wikitext_2+"train.jsonl"
valid = linked_wikitext_2+"valid.jsonl"
test = linked_wikitext_2+"test.jsonl"
synthetic_data = "generate_test_data/sythetic_dataset.jsonl"

data_files = {"train": train, "valid": valid, "test": test}
wikitest2_dataset = load_dataset("json", data_files=data_files)
synthetic_dataset = load_dataset("json", data_files={"synthetic": synthetic_data})
    
chunk_size = 128

class BertTokenizerModified(DistilBertTokenizer): #BertTokenizer
    def __init__(self,vocab_file,**kwargs):
        
        super().__init__(vocab_file, never_split=["@@START@@", "@@END@@", "@@start@@", "@@end@@"], **kwargs)
    
        self.tokenized_list = []

    def _tokenize(self, text):
        token_list = text.split()
        split_tokens = []
        tokenized_list = []
        
        if self.do_basic_tokenize:
            for token in token_list:

                # If the token is part of the never_split set
                if token in self.basic_tokenizer.never_split:
                    split_tokens.append(token)
                    tokenized_list.append(1)
                else:
                    word_tokenized = self.wordpiece_tokenizer.tokenize(token)
                    split_tokens += word_tokenized
                    tokenized_list.append(len(word_tokenized))

        self.tokenized_list.append(tokenized_list)
        return split_tokens
    
def get_cumm(vals):
    cumm = 0
    res = [0] ## len of res is 1 more than vals, with an initial 0
    for val in vals:
        cumm += val
        res.append(cumm)
    return res

def my_tokenize_function(data):
    
    ## tokenize
    my_tokenizer.tokenized_list = []
    result = my_tokenizer([" ".join(eg) for eg in data["tokens"]])
    if my_tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    
    ## save word to token mapping
    ## 3, 1, 1 means the first word got divided into 3 tokens, the next into 1, and the next into 1 again
    result["word_tokens"] = my_tokenizer.tokenized_list
    result["cummulative_word_tokens"] = [get_cumm(x) for x in result["word_tokens"]]
    
    return result

def get_kg_embedding_batched(data):
    
    ## store a masking array that says whether or not an item has kg embedding
    """
    When you specify batched=True the function receives a dictionary with the fields of the dataset, 
    but each value is now a list of values, and not just a single value. 
    """
    input_ids_list = data["input_ids"]
    annotations_list = data['annotations']
    cummulative_word_tokens_list = data["cummulative_word_tokens"]
    
    batch_size = len(input_ids_list)
    embed_list = [] ## len will be batch_size
    embed_mask = []
    embed_mask_qid = []
    
    for i in range(batch_size):
        input_ids = input_ids_list[i]
        annotations = annotations_list[i]
        
        ## Replace zeros with random numbers if required
        embeds = np.zeros((len(input_ids), 200)) ## CLS, SEP will have np.zeros, like unknown words
        mask = [0]*len(input_ids)
        mask_qid = ['0']*len(input_ids)
        
        for annot in annotations:
            start_ix, end_ix = annot['span']
            start = cummulative_word_tokens_list[i][start_ix] + 1
            end = cummulative_word_tokens_list[i][end_ix] + 1
            
            qid = annot['id']
            df = embeds_wktxt[embeds_wktxt['id']==qid]
            if len(df)>0:
                embeds[start:end] = np.tile(df.iloc[0,1:].values.reshape((1,200)),(end-start, 1))
                mask[start:end] = [1]*(end-start)
                mask_qid[start:end] = [qid]*(end-start)
                
        embed_mask.append(mask)
        embed_mask_qid.append(mask_qid)
        embed_list.append(embeds)

    return {
        "kg_embedding": embed_list, 
        "kg_embedding_mask": embed_mask,
        "kg_embedding_mask_qid": embed_mask_qid
    }

def filter_text_batched(data):
    
    new_data = {k:[] for k in data}
    
    input_ids_list = data["input_ids"]
    
    ## remove [UNK] == 100 
    indices_list = [[i for i,input_id in enumerate(input_ids) if input_id!=100]
                        for input_ids in input_ids_list]
    
    for k in data:
        for indices, data_list in zip(indices_list, data[k]):
            new_data[k].append([data_list[ind] for ind in indices])
        
    return new_data

def truncate_data(data):
    maxlength = my_tokenizer.max_model_input_sizes[bert_model_name]

    ## truncate to maxlength
    for k in data:
        data[k] = [x[:maxlength] for x in data[k]]
    
    return data


def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    
    # Create a new labels column
#     result["labels"] = result["input_ids"].copy()
    return result

Using custom data configuration default-c5571b3a8bc0c3d4
Found cached dataset json (/home/a2khatun/.cache/huggingface/datasets/json/default-c5571b3a8bc0c3d4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default-b38e982ae37a4744


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/a2khatun/.cache/huggingface/datasets/json/default-b38e982ae37a4744/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
my_tokenizer = BertTokenizerModified.from_pretrained(bert_model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerModified'.


In [30]:
dataset_file = "concat_dataset_v2"

In [ ]:
final_dataset = wikitest2_dataset.map(my_tokenize_function, batched=True)\
                          .map(get_kg_embedding_batched, batched=True, batch_size=100, keep_in_memory=False)\
                          .remove_columns(['title', 'tokens', 'annotations', 'word_tokens', 'cummulative_word_tokens'])\
                          .map(filter_text_batched, batched=True, batch_size=100, keep_in_memory=False)\
                          .map(group_texts, batched=True, batch_size=100, keep_in_memory=False)\

final_dataset.save_to_disk(dataset_file)

In [31]:
## Load the saved tokenized dataset
final_dataset = load_from_disk(dataset_file)

In [32]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 17528
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 1798
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 2069
    })
})

In [13]:
dataset_file = "tokenized_synthetic_dataset"

tokenized_synthetic_dataset = synthetic_dataset.map(my_tokenize_function, batched=True)\
                          .map(get_kg_embedding_batched, batched=True, batch_size=100, keep_in_memory=False)\
                          .remove_columns(['title', 'tokens', 'annotations', 'word_tokens', 'cummulative_word_tokens'])\
#                           .map(filter_text_batched, batched=True, batch_size=100, keep_in_memory=False)\
#                           .map(group_texts, batched=True, batch_size=100, keep_in_memory=False)\

tokenized_synthetic_dataset.save_to_disk(dataset_file)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [14]:
tokenized_synthetic_dataset

DatasetDict({
    synthetic: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 500
    })
})

In [ ]:
for i in range(10):
    for key in tokenized_synthetic_dataset["synthetic"].features:
        print(tokenized_synthetic_dataset["synthetic"][key][i])

In [17]:
## Create Data Collator for Masking

from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from collections.abc import Mapping

class CustomDataCollator(DataCollatorForWholeWordMask):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        if isinstance(examples[0], Mapping):
            input_ids = [e["input_ids"] for e in examples]
            kg_embedding_mask = [e["kg_embedding_mask"] for e in examples]
            kg_embedding = [e["kg_embedding"] for e in examples]
            kg_embedding_mask_qid = [e["kg_embedding_mask_qid"] for e in examples]
        else:
            raise Exception("Dataset needs to be in dictionary format")

        batch_input = _torch_collate_batch(input_ids, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)

        mask_labels = kg_embedding_mask
        batch_mask = _torch_collate_batch(mask_labels, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
        inputs, labels = self.torch_mask_tokens(batch_input, batch_mask)
        
        return {
                "input_ids": inputs, 
                "labels": labels, 
                "kg_embedding":kg_embedding, 
                "kg_embedding_mask":kg_embedding_mask,
                "kg_embedding_mask_qid":kg_embedding_mask_qid,
               }
    

# Create Model

In [18]:
from transformers import PreTrainedModel

class BERTModified(nn.Module): #PreTrainedModel
    def __init__(self, bert_model_name, base_model, config):
        
#         super().__init__(config) #For PreTrainedModel
        super().__init__() ## for nn.Module
        
        self.base_model = base_model
        self.config = config
                
        self.activation = GELUActivation() # for distilbert
        self.vocab_transform = nn.Linear(self.config.dim, self.config.dim)
        self.vocab_layer_norm = nn.LayerNorm(self.config.dim, eps=1e-12)
        self.vocab_projector = nn.Linear(self.config.dim, self.config.vocab_size)

        self.mlm_loss_fct = nn.CrossEntropyLoss()
        
        ## set to eval
        self.base_model.eval()
        
        ## freeze model
        for param in self.base_model.parameters():
            param.requires_grad = False

    def forward(
        self,
        kg_embedding = None,           ## given
        kg_embedding_mask = None,      ## given
        kg_embedding_mask_qid = None,      ## given
        input_ids = None,              ## given
        attention_mask = None,         ## given
        head_mask = None,
        inputs_embeds = None,
        labels = None,                 ## given
        output_attentions = None,
        output_hidden_states = None,
        return_dict= None,):
        
        base_model_output = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        ## Get LM embedding
        hidden_states = base_model_output[0]  # (bs, seq_length, dim)
        
        ## TODO: Use hidden_states and kg_embedding and perform INTEGRATION
        prediction_logits = self.vocab_transform(hidden_states)  # (bs, seq_length, dim)
        prediction_logits = self.activation(prediction_logits)  # (bs, seq_length, dim)
        prediction_logits = self.vocab_layer_norm(prediction_logits)  # (bs, seq_length, dim)
        prediction_logits = self.vocab_projector(prediction_logits)  # (bs, seq_length, vocab_size)

        mlm_loss = None
        if labels is not None:
            mlm_loss = self.mlm_loss_fct(prediction_logits.view(-1, prediction_logits.size(-1)), labels.view(-1))

        return MaskedLMOutput(
            loss=mlm_loss,
            logits=prediction_logits,
            hidden_states=base_model_output.hidden_states,
            attentions=base_model_output.attentions,
        )

In [19]:
base_model = AutoModel.from_pretrained(bert_model_name)
BERTModified_model = BERTModified(bert_model_name = bert_model_name,
                                  base_model = base_model,
                                  config = base_model.config)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# BERTModified_model = AutoModel.from_pretrained("Aisha/BERTModified-finetuned-wikitext-test")

In [21]:
data_collator = CustomDataCollator(tokenizer=my_tokenizer, mlm=True, mlm_probability=0.15)
# DataCollatorForWholeWordMask(tokenizer=my_tokenizer, mlm=True, mlm_probability=0.15)

In [33]:
# To test model with smaller sample dataset

train_size = 100
test_size = 10

downsampled_dataset = final_dataset["train"].train_test_split(train_size=train_size, test_size=test_size, seed=42)
downsampled_dataset

Loading cached split indices for dataset at concat_dataset_v2/train/cache-4211a33408ac86a5.arrow and concat_dataset_v2/train/cache-720f0802bdea057f.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 10
    })
})

In [34]:
import evaluate

# metrics = evaluate.combine(["accuracy", "precision", "recall", "f1"])
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_preds=None, logits=None, labels=None):
    
    # We should have either `eval_preds` or both `logits` and `labels`
    if eval_preds:
        logits, labels = eval_preds

    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[l for l in label if l != -100] for label in labels]
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    ## Flatten values
    true_labels = [item for sublist in true_labels for item in sublist]
    true_predictions = [item for sublist in true_predictions for item in sublist]
    
    accuracy = accuracy_metric.compute(predictions=true_predictions, references=true_labels)
    precision = precision_metric.compute(predictions=true_predictions, references=true_labels, average="micro")
    recall = recall_metric.compute(predictions=true_predictions, references=true_labels, average="micro")
    f1 = f1_metric.compute(predictions=true_predictions, references=true_labels, average="micro")
    
    return {
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
        "accuracy": accuracy["accuracy"],
    }

In [35]:
from transformers import TrainingArguments

batch_size = 4

# Show the training loss with every epoch
logging_steps = len(downsampled_dataset['train']) // batch_size #len(final_dataset['train']) // batch_size
model_name = "BERTModified"
output_dir = f"{model_name}-finetuned-wikitext-test"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
#     fp16=True,
    logging_steps=logging_steps,
    num_train_epochs=1,
#     load_best_model_at_end=True,
#     metric_for_best_model="loss",#metric_name,
#     greater_is_better = False,
    logging_dir='logs',
    report_to="wandb",
#     no_cuda=True,
)

/home/a2khatun/.venv/analysis/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


metric_for_best_model (str, optional) — Use in conjunction with load_best_model_at_end to specify the metric to use to compare two different models. Must be the name of a metric returned by the evaluation with or without the prefix "eval_". Will default to "loss" if unspecified and load_best_model_at_end=True (to use the evaluation loss).

If you set this value, greater_is_better will default to True. Don’t forget to set it to False if your metric is better when lower.

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=BERTModified_model,
    args=training_args,
    train_dataset=downsampled_dataset["train"], #final_dataset["train"],
    eval_dataset=downsampled_dataset["test"], #final_dataset["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/a2khatun/Downloads/KG/project/BERTModified-finetuned-wikitext-test is already a clone of https://huggingface.co/Aisha/BERTModified-finetuned-wikitext-test. Make sure you pull the latest changes with `repo.git_pull()`.


In [30]:
# samples = [trainer.eval_dataset[i] for i in range(2)]
# dc = data_collator(samples)
# kg_embedding_mask = [sample['kg_embedding_mask'] for sample in samples]

# for id_list, label_list, mask_list in zip(dc["input_ids"], dc["labels"], kg_embedding_mask):
#     tokens = my_tokenizer.convert_ids_to_tokens(id_list)
#     labels = my_tokenizer.convert_ids_to_tokens(label_list)
#     for token, label, mask in zip(tokens, labels, mask_list):
#         print(token, '\t', label, '\t', mask)
#     break

## Train Model

In [31]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 10
  Batch size = 4


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: tanny411. Use `wandb login --relogin` to force relogin


>>> Perplexity: 38112.84


In [32]:
eval_results

{'eval_loss': 10.548306465148926,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.9264,
 'eval_samples_per_second': 10.794,
 'eval_steps_per_second': 3.238}

# TODO: Trainer.model is not a `PreTrainedModel`, only saving its state dict.

In [33]:
trainer.train()
# trainer.save_model("output/models/BERTModified")

/home/a2khatun/.venv/analysis/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 25
  Number of trainable parameters = 24063546


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,10.461300,10.482824,0.000000,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 10
  Batch size = 4
Saving model checkpoint to BERTModified-finetuned-wikitext-test/checkpoint-25
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=25, training_loss=10.46131591796875, metrics={'train_runtime': 29.1341, 'train_samples_per_second': 3.432, 'train_steps_per_second': 0.858, 'total_flos': 0.0, 'train_loss': 10.46131591796875, 'epoch': 1.0})

In [34]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}") #21596 for 1 epoch

eval_results

***** Running Evaluation *****
  Num examples = 10
  Batch size = 4


>>> Perplexity: 36000.85


{'eval_loss': 10.491297721862793,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.9095,
 'eval_samples_per_second': 10.995,
 'eval_steps_per_second': 3.298,
 'epoch': 1.0}

In [ ]:
## Need to login to huggingface to push to hub

trainer.push_to_hub()

## Test model

In [37]:
## errors in GPU (then pc cant find GPU anymore)
predictions = trainer.predict(downsampled_dataset["test"]) #final_dataset["valid"] cpu crashes
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 10
  Batch size = 4


(10, 128, 30522) (10, 128)


In [37]:
preds = np.argmax(predictions.predictions, axis=-1)
preds

array([[ 8976, 18199,  8916, ..., 25969, 23858, 18199],
       [ 4463, 10208,  4463, ..., 22102,  4463, 17147],
       [17912, 13809,  7182, ..., 11629, 27497, 17912],
       ...,
       [21459,    54, 17240, ...,  5523, 12345, 13532],
       [14133,  1076, 19017, ...,  8466, 28247, 19711],
       [11487,  6334, 29788, ..., 14683,  6194, 28165]])

In [38]:
predictions.label_ids

array([[ -100,  -100,  -100, ...,  -100,  -100,  -100],
       [ -100,  -100,  -100, ...,  -100,  -100,  -100],
       [ -100,  -100,  -100, ...,  8540, 12256, 26775],
       ...,
       [ -100,  -100,  -100, ...,  -100,  -100,  -100],
       [15756,  6125,  -100, ...,  -100,  -100,  -100],
       [ -100,  -100,  -100, ...,  -100,  -100,  -100]])

[About micro, macro, weighted precision, recall, f1 for multiclass labels](https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1)

The following always holds true for the micro-F1 case:

`micro-F1 = micro-precision = micro-recall = accuracy`

In [38]:
compute_metrics(logits = predictions.predictions, labels = predictions.label_ids)
# metric.compute(predictions=preds, references=predictions.label_ids)

{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.0}

## Synthetic Data

In [44]:
train_size = 100
test_size = 10

downsampled_dataset2 = tokenized_synthetic_dataset["synthetic"].train_test_split(train_size=train_size, test_size=test_size, seed=42)
downsampled_dataset2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'kg_embedding', 'kg_embedding_mask', 'kg_embedding_mask_qid'],
        num_rows: 10
    })
})

In [45]:
predictions = trainer.predict(downsampled_dataset2["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 10
  Batch size = 4


(10, 24, 30522) (10, 24)


In [46]:
compute_metrics(logits = predictions.predictions, labels = predictions.label_ids)

{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.0}

## Try from API

In [ ]:
from transformers import pipeline

# Initialize MLM pipeline
mlm = pipeline('fill-mask', model=BERTModified_model, tokenizer=my_tokenizer)

# Get mask token
mask = mlm.tokenizer.mask_token

# Get result for particular masked phrase
phrase = f'Wikipedia is a free online {mask}, created and edited by volunteers around the world'

result = mlm(phrase)

# Print result
print(result)

In [ ]:
for x in result:
    print(f">>> {x['sequence']}")

In [ ]:
- Add a layer for KG embedding, add the losses
- new tokenizer for test data